# Reusable Verifiers 

This notebook demonstrates how to create and reuse the same set of separated verifiers for different models. Specifically, we will use the same verifier for the following four models:

- `1l_mlp sigmoid`
- `1l_mlp relu`
- `1l_conv sigmoid`
- `1l_conv relu`

When deploying EZKL verifiers on the blockchain, each associated model typically requires its own unique verifier, leading to increased on-chain state usage. 
However, with the reusable verifier, we can deploy a single verifier that can be used to verify proofs for any valid H2 circuit. This notebook shows how to do so. 

By reusing the same verifier across multiple models, we significantly reduce the amount of state bloat on the blockchain. Instead of deploying a unique verifier for each model, we deploy a unique and much smaller verifying key artifact (VKA) contract for each model while sharing a common separated verifier. The VKA contains the VK for the model as well circuit specific metadata that was otherwise hardcoded into the stack of the original non-reusable verifier.

In [ ]:
import torch
import torch.nn as nn
import torch.onnx

# Define the models
class MLP_Sigmoid(nn.Module):
    def __init__(self):
        super(MLP_Sigmoid, self).__init__()
        self.fc = nn.Linear(3, 3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

class MLP_Relu(nn.Module):
    def __init__(self):
        super(MLP_Relu, self).__init__()
        self.fc = nn.Linear(3, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)
        return x

class Conv_Sigmoid(nn.Module):
    def __init__(self):
        super(Conv_Sigmoid, self).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=3, stride=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv(x)
        x = self.sigmoid(x)
        return x

class Conv_Relu(nn.Module):
    def __init__(self):
        super(Conv_Relu, self).__init__()
        self.conv = nn.Conv1d(1, 1, kernel_size=3, stride=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

# Instantiate the models
mlp_sigmoid = MLP_Sigmoid()
mlp_relu = MLP_Relu()
conv_sigmoid = Conv_Sigmoid()
conv_relu = Conv_Relu()

# Dummy input tensor for mlp
dummy_input_mlp = torch.tensor([[-1.5737053155899048, -1.708398461341858, 0.19544155895709991]])
input_mlp_path = 'mlp_input.json'

# Dummy input tensor for conv
dummy_input_conv = torch.tensor([[[1.4124163389205933, 0.6938204169273376, 1.0664031505584717]]])
input_conv_path = 'conv_input.json'

In [ ]:
names = ['mlp_sigmoid', 'mlp_relu', 'conv_sigmoid', 'conv_relu']
models = [mlp_sigmoid, mlp_relu, conv_sigmoid, conv_relu]
inputs = [dummy_input_mlp, dummy_input_mlp, dummy_input_conv, dummy_input_conv]
input_paths = [input_mlp_path, input_mlp_path, input_conv_path, input_conv_path]

In [ ]:
import os
import json
import torch
import ezkl

for name, model, x, input_path in zip(names, models, inputs, input_paths):
    # Create a new directory for the model if it doesn't exist
    if not os.path.exists(name):
        os.mkdir(name)
    # Store the paths in each of their respective directories
    model_path = os.path.join(name, "network.onnx")
    compiled_model_path = os.path.join(name, "network.compiled")
    pk_path = os.path.join(name, "test.pk")
    vk_path = os.path.join(name, "test.vk")
    settings_path = os.path.join(name, "settings.json")

    witness_path = os.path.join(name, "witness.json")
    sol_code_path = os.path.join(name, 'test.sol')
    sol_key_code_path = os.path.join(name, 'test_key.sol')
    abi_path = os.path.join(name, 'test.abi')
    proof_path = os.path.join(name, "proof.json")

    # Flips the neural net into inference mode
    model.eval()

    # Export the model
    torch.onnx.export(model, x, model_path, export_params=True, opset_version=10,
                      do_constant_folding=True, input_names=['input'],
                      output_names=['output'], dynamic_axes={'input': {0: 'batch_size'},
                                                              'output': {0: 'batch_size'}})

    data_array = ((x).detach().numpy()).reshape([-1]).tolist()
    data = dict(input_data=[data_array])
    json.dump(data, open(input_path, 'w'))

    py_run_args = ezkl.PyRunArgs()
    py_run_args.input_visibility = "private"
    py_run_args.output_visibility = "public"
    py_run_args.param_visibility = "fixed"  # private by default

    res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
    assert res == True

    await ezkl.calibrate_settings(input_path, model_path, settings_path, "resources")

    res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
    assert res == True

    res = await ezkl.get_srs(settings_path)
    assert res == True

    # now generate the witness file
    res = await ezkl.gen_witness(input_path, compiled_model_path, witness_path)
    assert os.path.isfile(witness_path) == True

    # SETUP 
    # We recommend disabling selector compression for the setup as it decreases the size of the VK artifact
    res = ezkl.setup(compiled_model_path, vk_path, pk_path, disable_selector_compression=True)
    assert res == True
    assert os.path.isfile(vk_path)
    assert os.path.isfile(pk_path)
    assert os.path.isfile(settings_path)

    # GENERATE A PROOF
    res = ezkl.prove(witness_path, compiled_model_path, pk_path, proof_path, "single")
    assert os.path.isfile(proof_path)

    res = await ezkl.create_evm_verifier(vk_path, settings_path, sol_code_path, abi_path, reusable=True)
    assert res == True

    res = await ezkl.create_evm_vka(vk_path, settings_path, sol_key_code_path, abi_path)
    assert res == True


In [ ]:
import subprocess
import time

# make sure anvil is running locally
# $ anvil -p 3030

RPC_URL = "http://localhost:3030"

# Save process globally
anvil_process = None

def start_anvil():
    global anvil_process
    if anvil_process is None:
        anvil_process = subprocess.Popen(["anvil", "-p", "3030", "--code-size-limit=41943040"])
        if anvil_process.returncode is not None:
            raise Exception("failed to start anvil process")
        time.sleep(3)

def stop_anvil():
    global anvil_process
    if anvil_process is not None:
        anvil_process.terminate()
        anvil_process = None


Check that the generated verifiers are identical for all models.

In [ ]:
start_anvil()

In [ ]:
import filecmp

def compare_files(file1, file2):
    return filecmp.cmp(file1, file2, shallow=False)

sol_code_path_0 = os.path.join("mlp_sigmoid", 'test.sol')
sol_code_path_1 = os.path.join("mlp_relu", 'test.sol')

sol_code_path_2 = os.path.join("conv_sigmoid", 'test.sol')
sol_code_path_3 = os.path.join("conv_relu", 'test.sol')


assert compare_files(sol_code_path_0, sol_code_path_1) == True
assert compare_files(sol_code_path_2, sol_code_path_3) == True

Here we deploy separate verifier that will be shared by the four models. We picked the `1l_mlp sigmoid` model as an example but you could have used any of the generated verifiers since they are all identical. 

In [ ]:
import os 
addr_path_verifier = "addr_verifier.txt"
sol_code_path = os.path.join("mlp_sigmoid", 'test.sol')

res = await ezkl.deploy_evm(
    addr_path_verifier,
    sol_code_path,
    'http://127.0.0.1:3030',
    "verifier/reusable"
)

assert res == True

with open(addr_path_verifier, 'r') as file:
    addr = file.read().rstrip()

Finally we deploy each of the unique VK-artifacts and verify them using the shared verifier deployed in the previous step.

In [ ]:
for name in names:
    addr_path_vk = "addr_vk.txt"
    sol_key_code_path = os.path.join(name, 'test_key.sol')
    res = await ezkl.deploy_evm(addr_path_vk, sol_key_code_path, 'http://127.0.0.1:3030', "vka")
    assert res == True

    with open(addr_path_vk, 'r') as file:
        addr_vk = file.read().rstrip()
        
    proof_path = os.path.join(name, "proof.json")
    sol_code_path = os.path.join(name, 'vk.sol')
    res = await ezkl.verify_evm(
        addr,
        proof_path,
        "http://127.0.0.1:3030",
        addr_vk = addr_vk
    )
    assert res == True